In [6]:
import flywheel
import datetime
import pytz
import time 

# Create client
fw = flywheel.Client()

# define containers & files
gear = fw.lookup('gears/presurg/1.0.17')
project = fw.lookup('davis/presurgicalEpilepsy')

In [7]:
# create subject list
fin = 'patients.txt'
sid_list = [line.rstrip('\n').split()[0] for line in open(fin)]

In [8]:
print(sid_list)

['P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39', 'P40', 'P41', 'P42', 'P43', 'P44', 'P45', 'P46', 'P47', 'P48', 'P49', 'P50', 'P51', 'P52', 'P53', 'P54', 'P55', 'P56', 'P57', 'P58', 'P59', 'P60', 'P61', 'P62', 'P63', 'P64', 'P65', 'P66', 'P67', 'P68', 'P69', 'P70', 'P71', 'P72', 'P73', 'P74', 'P75', 'P76', 'P77', 'P78', 'P79', 'P80']


In [9]:
sid_list = ['P26','P27']

In [10]:
for sub in sid_list:
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    print('Trying subject %s' % (fw.get(ses.parents['subject']).label))

    # Check if there is task-fMRI data
    bold_img = None
    for acq in ses.acquisitions():
        if 'BOLD_Sentence' in acq.label or 'BOLD_Wordgen' in acq.label or 'BOLD_SceneMem' in acq.label:
            bold_container=fw.get(acq.id)
            for file in bold_container.files:
                if '.nii.gz' in file.name:
                    bold_img=file
    if bold_img is None:
        print("  %s: No task-fMRI data." % (sub))

    # Get the fmriprep analysis and check if already been run successfully
    for analysis in ses.analyses:
        if analysis.files and "fmriprep" in analysis.label:
            fprep = fw.get(analysis.id)
            print(analysis.label)
    for file in fprep.files:
        if "fmriprep" in file.name:
            fmriprepdir = file
            print(' Using %s as input fmriprep directory' % (fmriprepdir.name))

    # Run gear
    if fprep is not None and bold_img is not None:
        print(' Running analysis for %s' % (sub))
        inputs = {'fmriprepdir': fmriprepdir }
        config = {'AROMA': True, 'save_intermediary_files': True,'fwhm': 6, 'cluster_size_thresh': 150}
        analysis_id = gear.run(analysis_label='lingua-map (1)', config=config, inputs=inputs, destination=ses)

Trying subject P26
SB_fmriprep 8-13-2019 15:21
 Using fmriprep_sub-P26_5d530da8c3758a0047ddd5a9.zip as input fmriprep directory
 Running analysis for P26
Trying subject P27
SB_fmriprep 8-13-2019 15:21
 Using fmriprep_sub-P27_5d530da7c3758a0027d44824.zip as input fmriprep directory
 Running analysis for P27


In [ ]:
# sleep for an hour
#time.sleep(216000)

In [23]:
utc = pytz.UTC
threshold = datetime.datetime(2019, 11, 6, 10)
threshold = utc.localize(threshold)

# Retry failed analyses
for sub in sid_list[28:]:
    if sub=='P31' or sub=='P33' or sub=='P46' or sub=='P64': # fix these later
        continue
    ses = fw.lookup('davis/presurgicalEpilepsy/%s/ses-01' % (sub))
    for analysis in ses.analyses:
        if "lingua-map" in analysis.label and analysis.job.state=='failed' and (analysis.created > threshold):
            print("%s's job failed. Retrying..." % (sub))
            job = analysis.job
            print(job.attempt)
            print(job.failure_reason)
            new_id = fw.retry_job(job.id)
            print(new_id)


P32's job failed. Retrying...
1
{'_id': '5dc48224e74aa300601ad967'}
P39's job failed. Retrying...
1
{'_id': '5dc4822fe74aa3005e1105e5'}
P47's job failed. Retrying...
1
{'_id': '5dc4823ae74aa3005e1105eb'}
P53's job failed. Retrying...
1
{'_id': '5dc48245e74aa30057003012'}
P53's job failed. Retrying...
1
{'_id': '5dc48246e74aa3005e1105ed'}
P64's job failed. Retrying...
1
{'_id': '5dc48258e74aa300601ad992'}
P64's job failed. Retrying...
1
{'_id': '5dc48259e74aa3005e110616'}
P76's job failed. Retrying...
1
{'_id': '5dc4826ee74aa300601ada37'}
